In [1]:
import os
import openai
import pandas as pd
from p_tqdm import p_map
from tqdm.auto import tqdm
from dotenv import load_dotenv, find_dotenv

pd.set_option('display.max_colwidth', 512)
pd.set_option('display.max_columns', None)

class InferenceGPT():
    def __init__(self, model="gpt-3.5-turbo", temperature=0) -> None:
        _ = load_dotenv(find_dotenv())  # read local .env file
        openai.api_key = os.getenv('OPENAI_API_KEY')
        self.model = model
        self.temperature = temperature

    def run_prompts(self, prompts):
        raw_responses = p_map(self._get_completion, prompts, num_cpus=10, tqdm=tqdm)
        # raw_responses = [self._get_completion(p) for p in prompts]
        response = [r["final_content"] for r in raw_responses]
        return response, raw_responses

    def _get_completion(self, prompt):
        client = openai.OpenAI()
        response = client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            top_p=0,
            messages=[{"role": "user", "content": prompt}]
        )
        response = response.model_dump()  # ChatCompletion Class to dict (pydantic)
        response["final_content"] = response["choices"][0]["message"]["content"]
        return response

# MODEL = "gpt-4o-mini-2024-07-18"
MODEL = "gpt-4o-2024-08-06"
gpt = InferenceGPT(model=MODEL, temperature=0)

# Make Poison

In [2]:
df = pd.read_json("dataset/foil.jsonl", lines=True)
few_shot = """QUESTION: who won a million on deal or no deal
Answer: Tomorrow Rodriguez

QUESTION: When did the publisher of Tetrisphere unveil their new systems?
Answer: October 18, 1985

QUESTION: who is the woman washing the car in cool hand luke
Answer: Joy Harmon
"""
def prompt_template(query, doc=None):
    if doc is not None:
        return few_shot + f"DOCUMENT: {doc}\n\nQUESTION: {query}\n\nAnswer: "
    else:
        return few_shot + f"QUESTION: {query}\n\nAnswer: "

def flatten(xss):
    return [x for xs in xss for x in xs]

df["evidence_sentence"] = df["evidence_sents"].apply(lambda x: " ".join(flatten(x)))
df["foil_doc_prompt"] = df.apply(lambda row: prompt_template(row["query"], row["sents_complete_wo_foil"]), axis=1)
df["evidence_doc_prompt"] = df.apply(lambda row: prompt_template(row["query"], row["sents_complete_w_foil"]), axis=1)
df["query_prompt"] = df.apply(lambda row: prompt_template(row["query"]), axis=1)

df[["title", "query", "sents_complete_w_foil", "sents_complete_wo_foil", "relation_name", "tail_entity_names", "tail_entity_longest_name", "evidence_sents", "evidence_sentence"]]

replacements = [("10, ", "10 , "), ("22, ", "22 , "), ("15, ", "15 , "), ("13, ", "13 , "), ("17, ", "17 , "), ("Jazz", "jazz")]
for r in replacements:
    df["tail_entity_longest_name"] = df["tail_entity_longest_name"].str.replace(r[0], r[1])

df[["sents_complete_wo_foil", "tail_entity_longest_name"]]
for r in df.to_dict(orient="records"):
    if r["tail_entity_longest_name"] not in r["evidence_sentence"]:
        print(r["tail_entity_longest_name"], "|", r["evidence_sentence"])
        print(r)
        
df_raw = df.copy()

the United States | " How to Save a Life " is the twenty - first episode of the eleventh season of the American television medical drama Grey 's Anatomy , and is the 241st episode overall .
{'query': 'What is the country of origin of How to Save a Life?', 'gold_docs': ["How to Save a Life (Grey's Anatomy)"], 'gold_docs_text': {"How to Save a Life (Grey's Anatomy)": {'text': '" How to Save a Life " is the twenty - first episode of the eleventh season of the American television medical drama Grey \'s Anatomy , and is the 241st episode overall . It aired on April 23 , 2015 on ABC in the United States . The episode was written by showrunner Shonda Rhimes and directed by Rob Hardy , making it the first episode Rhimes has written since the season eight finale " Flight " . The installment marked the death and final appearance of the series’ male lead character , Derek Shepherd ( Patrick Dempsey ) , who had starred on the series since its inception . In this episode , Shepherd is involved in a

In [4]:
df = df_raw.copy()
# df = df.sample(1)

def poison_replace_prompt(tail, evidence):
    return f"In the sentence: '{evidence}', replace the entity '{tail}' with a different entity that makes sense in context but is completely different. Output only the replacement entity. replacement entity: "

df["poison_replace_prompt"] = df.apply(lambda row: poison_replace_prompt(row["tail_entity_longest_name"], row["evidence_sentence"]), axis=1)
df["poison_replace_prompt_output"] = gpt.run_prompts(df["poison_replace_prompt"])[0]
df["poison_doc"] = df.apply(lambda row: row["sents_complete_wo_foil"] + row["evidence_sentence"].replace(row["tail_entity_longest_name"], row["poison_replace_prompt_output"]), axis=1)
df["poison_doc_prompt"] = df.apply(lambda row: prompt_template(row["query"], row["poison_doc"]), axis=1)
df

  0%|          | 0/250 [00:00<?, ?it/s]

,query,gold_docs,gold_docs_text,scores_stats,scores_gold,predicted_docs_text_10,id,title,vertexSet,labels,sents,split,label,label_idx,head_entity,tail_entity,head_entity_names,tail_entity_names,head_entity_longest_name,tail_entity_longest_name,head_entity_types,tail_entity_types,evidence_sent_ids,evidence_sents,head_entity_in_evidence,tail_entity_in_evidence,relation,relation_name,query_question,duplicate_titles_len,duplicate_titles,hit_rank,gold_doc,gold_doc_title,gold_doc_text,gold_doc_score,pred_doc,pred_doc_title,pred_doc_text,pred_doc_score,gold_doc_len,pred_doc_len,sents_complete_w_foil,sents_complete_wo_foil,evidence_sentence,foil_doc_prompt,evidence_doc_prompt,query_prompt,poison_replace_prompt,poison_replace_prompt_output,poison_doc,poison_doc_prompt
0,When was The Private Life of Helen of Troy published?,[The Private Life of Helen of Troy],"{'The Private Life of Helen of Troy': {'text': 'The Private Life of Helen of Troy is a 1927 American silent film about Helen of Troy based on the 1925 novel of the same name by John Erskine , and adapted to screen by Gerald Duffy . The film was directed by Alexander Korda and starred María Corda as Helen , Lewis Stone as Menelaus , and Ricardo Cortez as Paris . Coming at the end of the silent film era , it was nominated for an Academy Award in 1929 , the year of the Awards ' inception , in the category ...","{'len': 1000, 'max': 392.3541870117, 'min': 373.9404907227, 'std': 1.3496554124, 'mean': 375.2744338684, 'median': 374.9059753418}",{'The Private Life of Helen of Troy': 392.3541870117},"{'The Private Life of Helen of Troy': {'text': 'The Private Life of Helen of Troy is a 1927 American silent film about Helen of Troy based on the 1925 novel of the same name by John Erskine , and adapted to screen by Gerald Duffy . The film was directed by Alexander Korda and starred María Corda as Helen , Lewis Stone as Menelaus , and Ricardo Cortez as Paris . Coming at the end of the silent film era , it was nominated for an Academy Award in 1929 , the year of the Awards ' inception , in the category ...",validation13053,The Private Life of Helen of Troy,"[[{'name': 'The Private Life of Helen of Troy', 'pos': [0, 7], 'sent_id': 0, 'type': 'MISC', 'global_pos': [0, 0], 'index': '0_0'}, {'name': 'The Private Life of Helen of Troy', 'pos': [21, 28], 'sent_id': 5, 'type': 'MISC', 'global_pos': [187, 187], 'index': '0_1'}, {'type': 'MISC', 'pos': [29, 36], 'name': 'The Private Life of Helen of Troy', 'sent_id': 4, 'global_pos': [149, 149], 'index': '0_2'}], [{'type': 'TIME', 'pos': [9, 10], 'name': '1927', 'sent_id': 0, 'global_pos': [9, 9], 'index': '1_0'}],...","[{'r': 'P577', 'h': 0, 't': 1, 'evidence': [0]}, {'r': 'P577', 'h': 0, 't': 5, 'evidence': [0]}, {'r': 'P57', 'h': 0, 't': 8, 'evidence': [1]}, {'r': 'P161', 'h': 0, 't': 9, 'evidence': [1]}, {'r': 'P161', 'h': 0, 't': 10, 'evidence': [1]}, {'r': 'P161', 'h': 0, 't': 12, 'evidence': [1]}, {'r': 'P58', 'h': 0, 't': 7, 'evidence': [0]}, {'r': 'P570', 'h': 7, 't': 17, 'evidence': [3]}, {'r': 'P571', 'h': 19, 't': 15, 'evidence': [2]}, {'r': 'P585', 'h': 16, 't': 15, 'evidence': [2]}, {'r': 'P31', 'h': 16, ...","[[The, Private, Life, of, Helen, of, Troy, is, a, 1927, American, silent, film, about, Helen, of, Troy, based, on, the, 1925, novel, of, the, same, name, by, John, Erskine, ,, and, adapted, to, screen, by, Gerald, Duffy, .], [The, film, was, directed, by, Alexander, Korda, and, starred, María, Corda, as, Helen, ,, Lewis, Stone, as, Menelaus, ,, and, Ricardo, Cortez, as, Paris, .], [Coming, at, the, end, of, the, silent, film, era, ,, it, was, nominated, for, an, Academy, Award, in, 1929, ,, the, year, o...",validation,"{'r': 'P577', 'h': 0, 't': 1, 'evidence': [0]}",0,"[{'name': 'The Private Life of Helen of Troy', 'pos': [0, 7], 'sent_id': 0, 'type': 'MISC', 'global_pos': [0, 0], 'index': '0_0'}, {'name': 'The Private Life of Helen of Troy', 'pos': [21, 28], 'sent_id': 5, 'type': 'MISC', 'global_pos': [187, 187], 'index': '0_1'}, {

In [5]:
# df.to_json("dataset/poison.jsonl", lines=True, orient="records")

# GPT RAG

In [2]:
df_raw = pd.read_json("dataset/poison.jsonl", lines=True)
df_raw.sample(1)

,query,gold_docs,gold_docs_text,scores_stats,scores_gold,predicted_docs_text_10,id,title,vertexSet,labels,sents,split,label,label_idx,head_entity,tail_entity,head_entity_names,tail_entity_names,head_entity_longest_name,tail_entity_longest_name,head_entity_types,tail_entity_types,evidence_sent_ids,evidence_sents,head_entity_in_evidence,tail_entity_in_evidence,relation,relation_name,query_question,duplicate_titles_len,duplicate_titles,hit_rank,gold_doc,gold_doc_title,gold_doc_text,gold_doc_score,pred_doc,pred_doc_title,pred_doc_text,pred_doc_score,gold_doc_len,pred_doc_len,sents_complete_w_foil,sents_complete_wo_foil,evidence_sentence,foil_doc_prompt,evidence_doc_prompt,query_prompt,poison_replace_prompt,poison_replace_prompt_output,poison_doc,poison_doc_prompt
150,Who is the spouse of Duryodhana?,[Kalinga (Mahabharata)],"{'Kalinga (Mahabharata)': {'text': 'Kalinga is a kingdom described in the legendary Indian text Mahabharata . Its location is the historical Kalinga region in present - day Odisha and Andhra Pradesh . Kuru prince Duryodhana 's wife Bhanumati was from Kalinga . Kalingas sided with Duryodhana in the Kurukshetra War . The founders of five eastern kingdoms , which included : Angas ( east , central Bihar ) , Vangas ( southern West Bengal and Bangladesh ) , Kalingas ( Sea shore of Odisha ) , Pundras ( western...","{'len': 1000, 'max': 390.1704711914, 'min': 379.3129882812, 'std': 1.3471653001, 'mean': 380.7124929504, 'median': 380.3125}",{'Kalinga (Mahabharata)': 384.7158813477},"{'Durdhara': {'text': 'Durdhara was the wife of Chandragupta Maurya , the founder of the 4th - century BCE Maurya Empire of ancient India , according to the 12th century CE Jain text Parishishtaparvan by Hemachandra . She is stated by this text to be the mother of the second Mauryan emperor , Bindusara . Nothing is mentioned or known about Durdhara outside of this legend written 1,600 years after Chandragupta 's era . Other sources , such as the Burmese Buddhist records do not corroborate the Jain legen...",test10501,Kalinga (Mahabharata),"[[{'name': 'Kalinga', 'pos': [0, 1], 'sent_id': 0, 'type': 'LOC', 'global_pos': [0, 0], 'index': '0_0'}, {'name': 'Kalinga', 'pos': [5, 6], 'sent_id': 1, 'type': 'LOC', 'global_pos': [17, 17], 'index': '0_1'}, {'name': 'Kalinga', 'pos': [8, 9], 'sent_id': 2, 'type': 'LOC', 'global_pos': [36, 36], 'index': '0_2'}, {'name': 'Kalinga', 'pos': [8, 9], 'sent_id': 5, 'type': 'LOC', 'global_pos': [115, 115], 'index': '0_3'}, {'name': 'Kalinga', 'pos': [18, 19], 'sent_id': 5, 'type': 'LOC', 'global_pos': [125, ...","[{'r': 'P26', 'h': 6, 't': 7, 'evidence': [2]}, {'r': 'P26', 'h': 7, 't': 6, 'evidence': [2]}, {'r': 'P131', 'h': 14, 't': 17, 'evidence': [4]}, {'r': 'P17', 'h': 14, 't': 17, 'evidence': [4]}, {'r': 'P150', 'h': 17, 't': 14, 'evidence': [4]}, {'h': 19, 't': 2, 'r': 'P1441', 'evidence': []}, {'h': 2, 't': 7, 'r': 'P674', 'evidence': []}, {'h': 3, 't': 17, 'r': 'P131', 'evidence': []}, {'h': 20, 't': 2, 'r': 'P1441', 'evidence': []}, {'h': 17, 't': 3, 'r': 'P150', 'evidence': []}, {'h': 12, 't': 17, 'r':...","[[Kalinga, is, a, kingdom, described, in, the, legendary, Indian, text, Mahabharata, .], [Its, location, is, the, historical, Kalinga, region, in, present, -, day, Odisha, and, Andhra, Pradesh, .], [Kuru, prince, Duryodhana, 's, wife, Bhanumati, was, from, Kalinga, .], [Kalingas, sided, with, Duryodhana, in, the, Kurukshetra, War, .], [The, founders, of, five, eastern, kingdoms, ,, which, included, :, Angas, (, east, ,, central, Bihar, ), ,, Vangas, (, southern, West, Bengal, and, Bangladesh, ), ,, Kali...",test,"{'r': 'P26', 'h': 6, 't': 7, 'evidence': [2]}",0,"[{'name': 'Duryodhana', 'pos': [2, 3], 'sent_id': 2, 'type': 'PER', 'global_pos': [30, 30], 'index': '6_0'}, {'name': 'Duryodhana', 'pos': [3, 4], 'sent_id': 3, 'type': 'PER', 'global_pos': [41, 41], 'index': '6_1'}]","[{'name': 'Bhanumati', 'pos': [5, 6], 'sent_id': 2, 'type': 'PER', 'global_pos': [33, 33], 'index': '7_0'}]",[Duryodhana],[

In [3]:
df = df_raw.copy()
# df = df.sample(5)

def eval_template(query, model_answer, gold_answer):
    return f"Query: {query}\nModel Answer: {model_answer}\n\nIs the model answer correct or not compared to the gold answer: {gold_answer} ? YES or NO\n\n"

for part in ["poison_doc", "foil_doc", "evidence_doc", "query"]:  # ["poison_doc", "foil_doc", "evidence_doc", "query"]
    df[f"{part}_prompt_output"] = gpt.run_prompts(df[f"{part}_prompt"])[0]
    df[f"{part}_prompt_eval_prompt"] = df.apply(lambda row: eval_template(row['query'], row[f"{part}_prompt_output"], row['tail_entity_longest_name']), axis=1)
    df[f"{part}_prompt_eval_prompt_output"] = gpt.run_prompts(df[f"{part}_prompt_eval_prompt"])[0]
    df[f"{part}_prompt_eval_prompt_output"] = df[f"{part}_prompt_eval_prompt_output"].apply(lambda s: s.upper().strip()[:3])

df["MODEL"] = MODEL
df.to_pickle(f"rag/eval_{MODEL}_{len(df)}.pkl")

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

In [4]:
df[["query", "evidence_doc_prompt_output", "evidence_doc_prompt_eval_prompt_output", "evidence_doc_prompt_eval_prompt",
    "foil_doc_prompt_output", "foil_doc_prompt_eval_prompt_output", "poison_doc_prompt_eval_prompt",]]

,query,evidence_doc_prompt_output,evidence_doc_prompt_eval_prompt_output,evidence_doc_prompt_eval_prompt,foil_doc_prompt_output,foil_doc_prompt_eval_prompt_output,poison_doc_prompt_eval_prompt
0,When was The Private Life of Helen of Troy published?,The Private Life of Helen of Troy was published in 1925.,NO,Query: When was The Private Life of Helen of Troy published?\nModel Answer: The Private Life of Helen of Troy was published in 1925.\n\nIs the model answer correct or not compared to the gold answer: 1927 ? YES or NO\n\n,The Private Life of Helen of Troy was published in 1925.,NO,"Query: When was The Private Life of Helen of Troy published?\nModel Answer: The novel ""The Private Life of Helen of Troy"" by John Erskine was published in 1925.\n\nIs the model answer correct or not compared to the gold answer: 1927 ? YES or NO\n\n"
1,What is EMH an instance of?,"EMH is an instance of a hologram, specifically the Emergency Medical Hologram, as depicted in the ""Star Trek"" franchise.",YES,"Query: What is EMH an instance of?\nModel Answer: EMH is an instance of a hologram, specifically the Emergency Medical Hologram, as depicted in the ""Star Trek"" franchise.\n\nIs the model answer correct or not compared to the gold answer: Doctor ? YES or NO\n\n",EMH is an instance of a hologram.,NO,Query: What is EMH an instance of?\nModel Answer: EMH is an instance of a hologram.\n\nIs the model answer correct or not compared to the gold answer: Doctor ? YES or NO\n\n
2,Which record label is God 's Son associated with?,Ill Will and Columbia Records,YES,Query: Which record label is God 's Son associated with?\nModel Answer: Ill Will and Columbia Records\n\nIs the model answer correct or not compared to the gold answer: Columbia Records ? YES or NO\n\n,Columbia Records,YES,Query: Which record label is God 's Son associated with?\nModel Answer: Ill Will and Def Jam Recordings\n\nIs the model answer correct or not compared to the gold answer: Columbia Records ? YES or NO\n\n
3,What is a notable work of Miami Sound Machine?,Eyes of Innocence and Primitive Love are notable works of Miami Sound Machine.,YES,Query: What is a notable work of Miami Sound Machine?\nModel Answer: Eyes of Innocence and Primitive Love are notable works of Miami Sound Machine.\n\nIs the model answer correct or not compared to the gold answer: Eyes of Innocence ? YES or NO\n\n,"""Dr. Beat"" is a notable work of Miami Sound Machine.",NO,"Query: What is a notable work of Miami Sound Machine?\nModel Answer: ""Rhythm Is Gonna Get You"" and ""Primitive Love"" are notable works of Miami Sound Machine.\n\nIs the model answer correct or not compared to the gold answer: Eyes of Innocence ? YES or NO\n\n"
4,Which country is Lake Ewauna associated with?,The United States.,YES,Query: Which country is Lake Ewauna associated with?\nModel Answer: The United States.\n\nIs the model answer correct or not compared to the gold answer: U.S. ? YES or NO\n\n,The United States.,YES,Query: Which country is Lake Ewauna associated with?\nModel Answer: United States\n\nIs the model answer correct or not compared to the gold answer: U.S. ? YES or NO\n\n
...,...,...,...,...,...,...,...
245,Which country is Bad Astronaut associated with?,Bad Astronaut is associated with the United States.,YES,Query: Which country is Bad Astronaut associated with?\nModel Answer: Bad Astronaut is associated with the United States.\n\nIs the model answer correct or not compared to the gold answer: American ? YES or NO\n\n,The band Bad Astronaut is associated with the United States.,YES,Query: Which country is Bad Astronaut associated with?\nModel Answer: The United States\n\nIs the model answer correct or not compared to the gold answer: American ? YES or NO\n\n
246,What conflict was Catinat part of?,Catinat was part of the conflict involving Prince Eugene of Savoy's campaign to seize the Spanish-controlled Duchy of Milan in the Italian peninsula.,NO,Query: What conflict was Catinat part of?\nModel Answer: Catinat was part

# Results

In [6]:
dfr = list()

paths = [
    "rag/eval_gpt-4o-mini-2024-07-18_250.pkl",
    "rag/eval_gpt-4o-2024-08-06_250.pkl",
]
for path in paths:
    df = pd.read_pickle(path)
    row = {"MODEL": df["MODEL"].iloc[0]}
    for part in ["poison_doc", "query", "foil_doc", "evidence_doc", ]:
        vc = df[f"{part}_prompt_eval_prompt_output"].value_counts()
        row[part] = vc.get("YES", 0) / vc.sum()
    dfr.append(row)

dfr = pd.DataFrame(dfr)
dfr

,MODEL,poison_doc,query,foil_doc,evidence_doc
0,gpt-4o-mini-2024-07-18,0.252,0.348,0.372,0.660
1,gpt-4o-2024-08-06,0.376,0.636,0.660,0.844


In [30]:
df["foil_doc_prompt_eval_prompt_output"].value_counts()

foil_doc_prompt_eval_prompt_output
NO     156
YES     94
Name: count, dtype: int64

In [31]:
df["query_prompt_eval_prompt_output"].value_counts()

query_prompt_eval_prompt_output
NO     157
YES     93
Name: count, dtype: int64